In [24]:
# Configuración del entorno
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Agregar el directorio raíz del proyecto al path para importar módulos
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))  # type: ignore

# Importar configuración y módulos del proyecto
from config import (  # type: ignore
    DATA_RAW_CSV_DIR,
    DATA_PROCESSED_DIR,
    OUTPUT_FIGURES_DIR,
    AÑOS_RANGO,
    TIPOS_DATOS
)
from scripts.data_cleaning import (
    cargar_tipo_normalizado,
    consolidar_todos_tipos,
    resumen_carga,
    validar_dataset,
    guardar_dataset
)



# Configuración de visualización
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['figure.dpi'] = 100

print("[OK] Entorno configurado correctamente")
print(f"[OK] Directorio de datos: {DATA_RAW_CSV_DIR}")
print(f"[OK] Años a procesar: {AÑOS_RANGO[0]}-{AÑOS_RANGO[1]-1}")
print(f"[OK] Tipos de datos: {TIPOS_DATOS}")

[OK] Entorno configurado correctamente
[OK] Directorio de datos: c:\Users\djlop\OneDrive\DIEGO\UVG\2026\primer semestre\Minería de Datos\Laboratorios\Lab 1\data\raw\csv
[OK] Años a procesar: 2009-2022
[OK] Tipos de datos: ['defunciones', 'defunciones_fetales', 'divorcios', 'matrimonios', 'nacimientos']


---

CONSOLIDACIÓN DE TODOS LOS TIPOS DE DATOS



In [25]:
# Cargar y consolidar todos los tipos de datos
print("Cargando archivos CSV de todos los tipos...")
print("="*80)

dfs_por_tipo, master_dataset = consolidar_todos_tipos(
    tipos_lista=TIPOS_DATOS,
    años_rango=AÑOS_RANGO,
    csv_dir=DATA_RAW_CSV_DIR,
    verbose=True
)

print("\n[OK] Consolidación completada")
print("="*80)

Cargando archivos CSV de todos los tipos...
CARGANDO Y NORMALIZANDO TODOS LOS TIPOS
✓ 2009: 96,001 filas, 28 columnas
✓ 2010: 96,001 filas, 28 columnas
✓ 2011: 96,001 filas, 28 columnas
✓ 2012: 72,657 filas, 28 columnas
✓ 2013: 72,657 filas, 28 columnas
✓ 2014: 77,807 filas, 28 columnas
✓ 2015: 80,876 filas, 29 columnas
✓ 2016: 80,876 filas, 29 columnas
✓ 2017: 3,033 filas, 31 columnas
✓ 2018: 2,894 filas, 30 columnas
✓ 2019: 85,600 filas, 28 columnas
✓ 2020: 96,001 filas, 28 columnas
✓ 2021: 96,001 filas, 28 columnas
✓ 2022: 95,386 filas, 28 columnas

DATASET: DEFUNCIONES
Total filas: 1,051,791
Total columnas: 50
Rango años: 2009 - 2022
Memoria: 557.67 MB

Columnas originales por año:
  2009: 27 columnas
  2010: 27 columnas
  2011: 27 columnas
  2012: 27 columnas
  2013: 27 columnas
  2014: 27 columnas
  2015: 28 columnas
  2016: 28 columnas
  2017: 30 columnas
  2018: 29 columnas
  2019: 27 columnas
  2020: 27 columnas
  2021: 27 columnas
  2022: 27 columnas

Completitud de datos (to

In [27]:
# Resumen de cada tipo de dato
print("\nRESUMEN INDIVIDUAL POR TIPO\n")
print("="*80)

for tipo in TIPOS_DATOS:
    if tipo in dfs_por_tipo:
        df = dfs_por_tipo[tipo]['df']  # Acceder al DataFrame dentro del diccionario
        print(f"\n{tipo.upper()}:")
        print(f"  - Filas: {len(df):,}")
        print(f"  - Columnas: {len(df.columns)}")
        print(f"  - Memoria: {df.memory_usage(deep=True).sum() / 1024**2:.1f} MB")
        print(f"  - Rango años: {df['año'].min()} - {df['año'].max()}")
        print(f"  - Nulos: {df.isnull().sum().sum():,} valores ({df.isnull().sum().sum() / df.size * 100:.2f}%)")
        
print("\n" + "="*80)


RESUMEN INDIVIDUAL POR TIPO


DEFUNCIONES:
  - Filas: 1,051,791
  - Columnas: 50
  - Memoria: 557.7 MB
  - Rango años: 2009 - 2022
  - Nulos: 21,910,972 valores (41.66%)

DEFUNCIONES_FETALES:
  - Filas: 38,213
  - Columnas: 35
  - Memoria: 15.1 MB
  - Rango años: 2009 - 2022
  - Nulos: 144,330 valores (10.79%)

DIVORCIOS:
  - Filas: 77,927
  - Columnas: 27
  - Memoria: 23.6 MB
  - Rango años: 2009 - 2022
  - Nulos: 483,451 valores (22.98%)

MATRIMONIOS:
  - Filas: 997,468
  - Columnas: 30
  - Memoria: 355.0 MB
  - Rango años: 2009 - 2022
  - Nulos: 6,250,861 valores (20.89%)

NACIMIENTOS:
  - Filas: 5,129,982
  - Columnas: 51
  - Memoria: 2375.6 MB
  - Rango años: 2009 - 2022
  - Nulos: 37,845,685 valores (14.47%)



In [28]:
# Resumen del dataset maestro
print("\nDATASET MAESTRO CONSOLIDADO\n")
print("="*80)
print(f"  - Total de registros: {len(master_dataset):,}")
print(f"  - Total de columnas: {len(master_dataset.columns)}")
print(f"  - Memoria total: {master_dataset.memory_usage(deep=True).sum() / 1024**2:.1f} MB")
print(f"  - Tipos incluidos: {master_dataset['tipo'].unique().tolist()}")
print(f"  - Rango temporal: {master_dataset['año'].min()} - {master_dataset['año'].max()}")
print("\nDistribución por tipo:")
print(master_dataset['tipo'].value_counts())
print("="*80)


DATASET MAESTRO CONSOLIDADO

  - Total de registros: 7,295,381
  - Total de columnas: 95
  - Memoria total: 6872.6 MB
  - Tipos incluidos: ['defunciones', 'defunciones_fetales', 'divorcios', 'matrimonios', 'nacimientos']
  - Rango temporal: 2009 - 2022

Distribución por tipo:
tipo
nacimientos            5129982
defunciones            1051791
matrimonios             997468
divorcios                77927
defunciones_fetales      38213
Name: count, dtype: int64


### Validación de Calidad de Datos

Antes de guardar, validamos que la normalización y consolidación se hayan realizado correctamente.

In [29]:
# VALIDACION 1: Verificar normalizacion de columnas
print("="*80)
print("VALIDACION: NORMALIZACION DE COLUMNAS")
print("="*80)

# Verificar que todas las columnas esten en lowercase
columnas_no_lower = [col for col in master_dataset.columns if col != col.lower()]

if columnas_no_lower:
    print(f"[ERROR] Columnas NO normalizadas encontradas: {columnas_no_lower}")
else:
    print("[OK] Todas las columnas estan en lowercase")

# Verificar que no haya duplicados por capitalizacion
columnas_lower = [col.lower() for col in master_dataset.columns]
duplicados = len(columnas_lower) - len(set(columnas_lower))

if duplicados > 0:
    print(f"[ERROR] Se encontraron {duplicados} columnas duplicadas por capitalizacion")
else:
    print(f"[OK] No hay columnas duplicadas (total: {len(master_dataset.columns)} columnas unicas)")

print("\n" + "="*80)

VALIDACION: NORMALIZACION DE COLUMNAS
[OK] Todas las columnas estan en lowercase
[OK] No hay columnas duplicadas (total: 95 columnas unicas)



In [31]:
# VALIDACION 2: Integridad del dataset maestro
print("="*80)
print("VALIDACION: INTEGRIDAD DE DATOS")
print("="*80)

# 1. Verificar que existan columnas criticas
columnas_criticas = ['año', 'tipo']
faltantes = [col for col in columnas_criticas if col not in master_dataset.columns]

if faltantes:
    print(f"[ERROR] Columnas criticas faltantes: {faltantes}")
else:
    print(f"[OK] Columnas criticas presentes: {columnas_criticas}")

# 2. Verificar rango de años (sin crear arrays grandes)
años_esperados = set(range(2009, 2023))
años_presentes = set(master_dataset['año'].unique())

if años_esperados == años_presentes:
    print(f"[OK] Todos los años presentes: {min(años_presentes)}-{max(años_presentes)}")
else:
    faltantes_años = años_esperados - años_presentes
    if faltantes_años:
        print(f"[ADVERTENCIA] Años faltantes: {sorted(faltantes_años)}")

# 3. Verificar tipos de datos
tipos_esperados = set(TIPOS_DATOS)
tipos_presentes = set(master_dataset['tipo'].unique())

if tipos_esperados == tipos_presentes:
    print(f"[OK] Todos los tipos presentes: {sorted(tipos_presentes)}")
else:
    faltantes_tipos = tipos_esperados - tipos_presentes
    if faltantes_tipos:
        print(f"[ERROR] Tipos faltantes: {sorted(faltantes_tipos)}")

# 4. Contar nulos por tipo sin cargar todo en memoria
total_nulos = master_dataset.isnull().sum().sum()
porcentaje_nulos = (total_nulos / (master_dataset.shape[0] * master_dataset.shape[1])) * 100
print(f"\n[INFO] Valores nulos: {total_nulos:,} ({porcentaje_nulos:.4f}% del dataset)")

print("\n" + "="*80)
print("[VALIDACION COMPLETADA]")
print("="*80)

VALIDACION: INTEGRIDAD DE DATOS
[OK] Columnas criticas presentes: ['año', 'tipo']
[OK] Todos los años presentes: 2009-2022
[OK] Todos los tipos presentes: ['defunciones', 'defunciones_fetales', 'divorcios', 'matrimonios', 'nacimientos']

[INFO] Valores nulos: 412,112,338 (59.4626% del dataset)

[VALIDACION COMPLETADA]


### Guardar Dataset Maestro

guardamos el dataset maestro consolidado en `data/processed/`.

In [ ]:
# ANALISIS DESCRIPTIVO POR TIPO - Reemplaza guardado maestro
print("[ANÁLISIS POR TIPO] inicio")
from pathlib import Path

processed_dir = Path(DATA_PROCESSED_DIR)
out_tables = Path('../output/tables')
out_figs = Path('../output/figures')
out_tables.mkdir(parents=True, exist_ok=True)
out_figs.mkdir(parents=True, exist_ok=True)

resumen_all = []
for tipo in TIPOS_DATOS:
    file_path = processed_dir / f"{tipo}_2009_2024.csv"
    if not file_path.exists():
        print(f"  {tipo}: archivo procesado no encontrado, saltando")
        continue
    df_tipo = pd.read_csv(file_path)
    print(f"\n{tipo.upper()}: {len(df_tipo):,} filas × {len(df_tipo.columns)} columnas")

    # columnas numéricas y estadísticas
    num_cols = df_tipo.select_dtypes(include=[np.number]).columns.tolist()
    if len(num_cols) == 0:
        print(f"    {tipo}: no hay columnas numéricas")
        continue

    stats = df_tipo[num_cols].describe().T
    stats_file = out_tables / f"stats_{tipo}.csv"
    stats.to_csv(stats_file)
    print(f"    Estadísticas guardadas: {stats_file.name}")

    # elegir variables para histogramas: top 6 por conteo
    top_vars = stats['count'].sort_values(ascending=False).head(6).index.tolist()
    for var in top_vars:
        serie = df_tipo[var].dropna()
        if serie.empty:
            continue
        plt.figure()
        sns.histplot(serie, bins=50, kde=False)
        plt.title(f"{tipo} — {var}")
        fig_file = out_figs / f"{tipo}_{var}_hist.png"
        plt.savefig(fig_file, dpi=150, bbox_inches='tight')
        plt.close()

    # agregar resumen por variable al consolidated
    for col in num_cols:
        s = df_tipo[col].dropna()
        if s.empty:
            continue
        resumen_all.append({
            'tipo': tipo,
            'columna': col,
            'count': int(s.count()),
            'mean': float(s.mean()),
            'std': float(s.std()),
            'min': float(s.min()),
            '25%': float(s.quantile(0.25)),
            '50%': float(s.quantile(0.5)),
            '75%': float(s.quantile(0.75)),
            'max': float(s.max())
        })

# exportar resumen consolidado
if resumen_all:
    resumen_df = pd.DataFrame(resumen_all)
    resumen_path = out_tables / '04_resumen_estadistico_por_tipo.csv'
    resumen_df.to_csv(resumen_path, index=False)
    print(f"\n  Resumen consolidado guardado: {resumen_path}")
else:
    print("\n  No se generó resumen consolidado (sin datos numéricos encontrados)")

print("[ANÁLISIS POR TIPO] completado")

[GUARDANDO] Exportando dataset maestro...


✓ Dataset maestro guardado en: c:\Users\djlop\OneDrive\DIEGO\UVG\2026\primer semestre\Minería de Datos\Laboratorios\Lab 1\data\processed\master_dataset.csv
  Tamaño: 1767.01 MB

[OK] Dataset maestro guardado en data/processed/

Para analizar tipos individuales, filtrar por columna 'tipo':
  Ejemplo: master[master['tipo'] == 'defunciones']


In [ ]:
import glob
import os


raw_path = '../data/raw/csv'
processed_path = '../data/processed'

tipos = ['defunciones', 'defunciones_fetales', 'divorcios', 'matrimonios', 'nacimientos']

print("Consolidando archivos por tipo...")
print("=" * 80)

for tipo in tipos:
    archivos = sorted(glob.glob(f'{raw_path}/*_{tipo}.csv'))
    
    if not archivos:
        print(f"  No se encontraron archivos para: {tipo}")
        continue
    
    dfs = []
    for archivo in archivos:
        año = os.path.basename(archivo).split('_')[0]
        
        if os.path.getsize(archivo) == 0:
            print(f"    {año}_{tipo}: archivo vacío (saltado)")
            continue
        
        try:
            df = pd.read_csv(archivo, dtype=str)
            df.columns = df.columns.str.lower()
            dfs.append(df)
            print(f"    {año}_{tipo}: {len(df):,} registros")
        except Exception as e:
            print(f"    {año}_{tipo}: error ({str(e)[:50]})")
            continue
    
    if not dfs:
        print(f"    No hay datos válidos para {tipo}")
        continue
    
    df_consolidado = pd.concat(dfs, ignore_index=True)
    
    cols_numericas = df_consolidado.select_dtypes(include='object').columns
    for col in cols_numericas:
        try:
            df_consolidado[col] = pd.to_numeric(df_consolidado[col], errors='coerce')
        except:
            pass
    
    output_file = f'{processed_path}/{tipo}_2009_2024.csv'
    df_consolidado.to_csv(output_file, index=False)
    print(f"  → Exportado: {tipo}_2009_2024.csv ({len(df_consolidado):,} registros)\n")

print("=" * 80)
print("  Consolidación completada")

Consolidando archivos por tipo...
  ✓ 2009_defunciones: 96,001 registros
  ✓ 2010_defunciones: 96,001 registros
  ✓ 2011_defunciones: 96,001 registros
  ✓ 2012_defunciones: 72,657 registros
  ✓ 2013_defunciones: 72,657 registros
  ✓ 2014_defunciones: 77,807 registros
  ✓ 2015_defunciones: 80,876 registros
  ✓ 2016_defunciones: 80,876 registros
  ✓ 2017_defunciones: 3,033 registros
  ✓ 2018_defunciones: 2,894 registros
  ✓ 2019_defunciones: 85,600 registros
  ✓ 2020_defunciones: 96,001 registros
  ✓ 2021_defunciones: 96,001 registros
  ✓ 2022_defunciones: 95,386 registros
  ✓ 2023_defunciones: 20 registros
  ✗ 2024_defunciones: error (No columns to parse from file)
  → Exportado: defunciones_2009_2024.csv (1,051,811 registros)

  ✓ 2009_defunciones_fetales: 2,306 registros
  ✓ 2010_defunciones_fetales: 2,306 registros
  ✓ 2011_defunciones_fetales: 2,306 registros
  ✓ 2012_defunciones_fetales: 3,157 registros
  ✓ 2013_defunciones_fetales: 3,157 registros
  ✓ 2014_defunciones_fetales: 3,2

: 